In [1]:
import os

In [2]:
os.environ["DGLBACKEND"] = "pytorch"

In [3]:
import torch 
import pandas as pd
import dgl
from dgl.data import DGLDataset
import dgl.function as fn
import numpy as np

In [4]:
ds = dgl.data.CSVDataset('./DGL_playground/toy_dataset/')

Done loading data from cached files.


In [5]:
g = ds[0]

In [37]:
g.edata['w'] = torch.rand(g.num_edges(),1)

In [38]:
g.edges("all")

(tensor([0, 0, 0, 1, 1, 2, 2, 3, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8]),
 tensor([1, 2, 3, 0, 2, 0, 1, 0, 4, 5, 3, 5, 3, 4, 7, 8, 6, 8, 6, 7]),
 tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19]))

In [39]:
g.edata['w']

tensor([[0.4685],
        [0.5463],
        [0.6963],
        [0.4754],
        [0.3563],
        [0.5243],
        [0.8724],
        [0.5312],
        [0.0729],
        [0.3145],
        [0.9544],
        [0.5473],
        [0.5350],
        [0.6529],
        [0.5015],
        [0.9761],
        [0.1826],
        [0.0806],
        [0.3927],
        [0.5473]])

The adjacency matrix method of dgl.sparse contains a bug which makes it impossible to retrieve the value of an edge property in the adjacency matrix, although the documentation does imply this ability. The folowing is a reimplementation of the method fixing that issue.

In [32]:
import torch
from dgl.sparse import spmatrix

def adj_mat_with_edge_prop(m, etype= None, eprop=None):
    
    
    etype = m.to_canonical_etype(etype)
    indices = torch.stack(m.all_edges(etype=etype))
    shape = (m.num_nodes(etype[0]), m.number_of_nodes(etype[2]))
    if eprop is not None:
        val = m.edges[etype].data[eprop].flatten()
    else:
        val = None
        
    return spmatrix(
        indices,
        val=val,
        shape=shape,
    )

Construct an adjacency matrix of graph `g` with the `w` edge values as entry

In [ ]:
adje = adj_mat_with_edge_prop(g, etype= None, eprop='w')

In [41]:
adje.to_dense()

tensor([[0.0000, 0.4685, 0.5463, 0.6963, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.4754, 0.0000, 0.3563, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.5243, 0.8724, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.5312, 0.0000, 0.0000, 0.0000, 0.0729, 0.3145, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.9544, 0.0000, 0.5473, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.5350, 0.6529, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5015, 0.9761,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1826, 0.0000, 0.0806,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3927, 0.5473, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000]])

In [33]:
#select the upper triangular matrix from a sparse matrix
def triu(spm):
    mask = spm.row <= spm.col
    return dgl.sparse.from_coo(spm.row[mask],spm.col[mask],spm.val[mask],shape=spm.shape)

In [34]:
#create a symmetrical matrix based on an input upper triangular matrix. Note, this works because the diagonal is zero as we have no self-loops  
def sym_from_triu(triuspm):
    return triuspm + triuspm.T

In [35]:
#symmetrize graph weights in place using triangular symmetrization as above
def symmetrize_edge_prop(g,etype=None,eprop=None):
    am = adj_mat_with_edge_prop(g, etype=etype, eprop=eprop)
    sam = sym_from_triu(triu(am))
    g.edata['w'] = sam.val
    
    
    
    

symmetrize the graph edge weights on the basis of the upper triangular matrix

In [42]:
symmetrize_edge_prop(g,eprop='w')

SparseMatrix(indices=tensor([[0, 0, 0, 1, 1, 2, 2, 3, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8],
                             [1, 2, 3, 0, 2, 0, 1, 0, 4, 5, 3, 5, 3, 4, 7, 8, 6, 8, 6, 7]]),
             values=tensor([0.4685, 0.5463, 0.6963, 0.4754, 0.3563, 0.5243, 0.8724, 0.5312, 0.0729,
                            0.3145, 0.9544, 0.5473, 0.5350, 0.6529, 0.5015, 0.9761, 0.1826, 0.0806,
                            0.3927, 0.5473]),
             shape=(10, 10), nnz=20)
SparseMatrix(indices=tensor([[0, 0, 0, 1, 1, 2, 2, 3, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8],
                             [1, 2, 3, 0, 2, 0, 1, 0, 4, 5, 3, 5, 3, 4, 7, 8, 6, 8, 6, 7]]),
             values=tensor([0.4685, 0.5463, 0.6963, 0.4685, 0.3563, 0.5463, 0.3563, 0.6963, 0.0729,
                            0.3145, 0.0729, 0.5473, 0.3145, 0.5473, 0.5015, 0.9761, 0.5015, 0.0806,
                            0.9761, 0.0806]),
             shape=(10, 10), nnz=20)


In [43]:
g.edata['w']

tensor([0.4685, 0.5463, 0.6963, 0.4685, 0.3563, 0.5463, 0.3563, 0.6963, 0.0729,
        0.3145, 0.0729, 0.5473, 0.3145, 0.5473, 0.5015, 0.9761, 0.5015, 0.0806,
        0.9761, 0.0806])

In [47]:
sadje = adj_mat_with_edge_prop(g,eprop='w')

In [48]:
sadje.to_dense()

tensor([[0.0000, 0.4685, 0.5463, 0.6963, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.4685, 0.0000, 0.3563, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.5463, 0.3563, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.6963, 0.0000, 0.0000, 0.0000, 0.0729, 0.3145, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0729, 0.0000, 0.5473, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.3145, 0.5473, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5015, 0.9761,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5015, 0.0000, 0.0806,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9761, 0.0806, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000]])